In [1]:
import os
%pwd

'/home/user/shahabas/MLops/end-to-end--ml-pipeline/research'

In [2]:
os.chdir("../")

In [3]:
pwd

'/home/user/shahabas/MLops/end-to-end--ml-pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [12]:
@dataclass(frozen=True)
class dataingestion_config:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path


In [13]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml,create_directories

In [23]:
class ConfiguarationManager:
    def __init__(
        self,
        config_file = config_file_path,
        schema_file = schema_file_path,
        params_file = params_file_path):

        self.config = read_yaml(config_file)
        self.schema = read_yaml(schema_file)
        self.params = read_yaml(params_file)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->dataingestion_config:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = dataingestion_config(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config


In [24]:
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: dataingestion_config):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [27]:
try:
    config = ConfiguarationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-15 08:49:37,700 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2025-04-15 08:49:37,712 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-04-15 08:49:37,716 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-04-15 08:49:37,718 : INFO : common : created directory at: artifacts]
[2025-04-15 08:49:37,720 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-04-15 08:49:39,156 : INFO : 1833745355 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D72E:2B8DF:1A0638C:1D87626:67FDF372
Accept-Ranges: 